In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../') #Esto sube un nivel desde 'Notebooks' a 'Proyect_amazon'(donde esta Functions.py)
import sys
# permite que Python encuentre e importe el módulo 'Functions' desde el directorio actual
sys.path.append(os.getcwd())
import Functions as fs

In [2]:
df_ventas = pd.read_csv('Amazon Sale Report.csv', low_memory=False)
df_ventas.head(2)

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN


In [3]:
df_ventas.shape

(128975, 24)

In [4]:
fs.data_characteristics(df_ventas)

,column,type data,total nulls,percentage
0,index,int64,0,0.0%
1,Order ID,object,0,0.0%
2,Date,object,0,0.0%
3,Status,object,0,0.0%
4,Fulfilment,object,0,0.0%
5,Sales Channel,object,0,0.0%
6,ship-service-level,object,0,0.0%
7,Style,object,0,0.0%
8,SKU,object,0,0.0%
9,Category,object,0,0.0%


**Eliminacion de columnas**

Se comenzara eliminando las columnas irrelevantes del conjunto de datos según lo mencionado en la conclusión del análisis preliminar.

In [5]:
columnas_innecesarias = ['index','promotion-ids','fulfilled-by','Unnamed: 22']
df_ventas.drop(columnas_innecesarias,axis=1,inplace=True)

In [6]:
#Se elimina la columna 'Courier Status' porque contiene la misma data que 'Status'
df_ventas.drop(['Courier Status'],axis=1,inplace=True)

In [7]:
df_ventas.dtypes

Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
ASIN                   object
Qty                     int64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-postal-code      float64
ship-country           object
B2B                      bool
dtype: object

In [8]:
#Cambio tipos de datos de la columna Date y ship-postal-code
df_ventas['Date'] = pd.to_datetime(df_ventas['Date'],format="%m-%d-%y", errors='coerce')
df_ventas['ship-postal-code'] = df_ventas['ship-postal-code'].astype('object')

#Se transformaron los valores a cadena (.astype(str)) para poder remplazar, porque sino da error
df_ventas['ship-postal-code'] = df_ventas['ship-postal-code'].astype(str).str.replace('.0', '')

In [9]:
df_ventas.sample()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,B2B
48716,171-3210086-9943507,2022-04-01,Shipped,Amazon,Amazon.in,Expedited,J0344,J0344-TP-M,Top,M,B0986XJDJ4,1,INR,499.0,SAGAR,KARNATAKA,577401,IN,False


**Modificacion de columnas**

In [10]:
# Elimino los espacios en blanco al inicio y al final de los nombres de las columnas
df_ventas.columns = df_ventas.columns.str.strip()

# formato snake case
new_names = { 'Order ID': 'order_id', 'Date': 'date', 'Status': 'shipping_status', 'Fulfilment': 'fulfilment', 
                'Sales Channel': 'sales_channel', 'ship-service-level': 'ship_service_level',  'Style': 'style', 
                'SKU': 'sku', 'Category': 'category', 'Size': 'size', 'ASIN': 'asin', 'Qty': 'quantity', 
                'Amount': 'amount', 'ship-city': 'ship_city', 'ship-state': 'ship_state', 'ship-postal-code': 'ship_postal_code', 
                'ship-country': 'ship_country', 'B2B': 'b2b'
}
#renombro las columnas
df_ventas.rename(columns=new_names,inplace=True)

In [11]:
df_ventas['shipping_status'].unique()

array(['Cancelled', 'Shipped - Delivered to Buyer', 'Shipped',
       'Shipped - Returned to Seller', 'Shipped - Rejected by Buyer',
       'Shipped - Lost in Transit', 'Shipped - Out for Delivery',
       'Shipped - Returning to Seller', 'Shipped - Picked Up', 'Pending',
       'Pending - Waiting for Pick Up', 'Shipped - Damaged', 'Shipping'],
      dtype=object)

In [12]:
#Se remplaza el valor 'Shipping' por 'Shipped' en la columna 'shipping_status'
df_ventas['shipping_status']=df_ventas['shipping_status'].str.replace('Shipping','Shipped')

In [13]:
# Divide la columna 'shipping_status' en 2 partes apartir del separador ' - '
split_columns = df_ventas['shipping_status'].str.split(' - ', expand=True)

# Asignar la primera parte a 'shipping_status' y la segunda parte a 'delivery_status'
df_ventas['shipping_status'] = split_columns[0]
df_ventas['delivery_status'] = split_columns[1]

columnas = list(df_ventas.columns)
#print(columnas)
nuevo_orden = ['order_id', 'date', 'shipping_status', 'delivery_status', 'fulfilment', 'sales_channel', 'ship_service_level', 'style', 'sku', 'category', 'size', 'asin', 'quantity', 'currency', 'amount', 'ship_city', 'ship_state', 'ship_postal_code', 'ship_country', 'b2b']

#Modifico el orden dejando 'delivery_status' al lado de 'shipping_status'
df_ventas = df_ventas.reindex(columns=nuevo_orden)

In [14]:
df_ventas['delivery_status'].isnull().sum()

96802

<small > Luego de dividir la columna *shipping_status* en 2, quedaron gran cantidad de valores nulos en la nueva columna *delivery_status* de acuerdo a los datos de *shipping_status* imputaremos de la siguiente forma:<br>

- Donde *shipping_status* = " cancelled " ira " cancelled " en *delivery_status*.
- Donde *shipping_status* = " Pending " ira " Pending " en *delivery_status*.
- Donde *shipping_status* = " Shipped " ira " on the way " en *delivery_status*.

La razon por la que habian quedado valores nulos se debe a que algunos valores de la columna *shipping_status* eran unicos (no estaban separados por - ) </small>

In [15]:
filtros = [
    (df_ventas['shipping_status'] == 'Cancelled') & (df_ventas['delivery_status'].isnull()),
    (df_ventas['shipping_status'] == 'Shipped') & (df_ventas['delivery_status'].isnull() ),
    (df_ventas['shipping_status'] == 'Pending') & (df_ventas['delivery_status'].isnull() )
]

# Valores a asignar según los filtros
valores = ['Cancelled', 'On the way', 'Pending']

# np.select decide qué valor tomar para 'delivery_status' revisando cada filtro en orden. Cuando encuentra 
# un filtro que sea verdadero para una fila específica, asigna el valor correspondiente de 'valores'
df_ventas['delivery_status'] = np.select(filtros, valores, default=df_ventas['delivery_status'])


In [16]:
df_ventas['delivery_status'].unique()

array(['Cancelled', 'Delivered to Buyer', 'On the way',
       'Returned to Seller', 'Rejected by Buyer', 'Lost in Transit',
       'Out for Delivery', 'Returning to Seller', 'Picked Up', 'Pending',
       'Waiting for Pick Up', 'Damaged'], dtype=object)

**Eliminacion de duplicados**

In [17]:
df_ventas.duplicated().sum()

6

In [18]:
# Identificar filas duplicadas considerando todas las columnas
duplicated_rows = df_ventas[df_ventas.duplicated(keep=False)]
duplicated_rows

,order_id,date,shipping_status,delivery_status,fulfilment,sales_channel,ship_service_level,style,sku,category,size,asin,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,b2b
30660,406-0372545-6086735,2022-04-12,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,SET197,SET197-KR-NP-L,Set,L,B08B3YPD63,0,NaN,NaN,Siliguri,WEST BENGAL,734008,IN,False
30661,406-0372545-6086735,2022-04-12,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,SET197,SET197-KR-NP-L,Set,L,B08B3YPD63,0,NaN,NaN,Siliguri,WEST BENGAL,734008,IN,False
41291,408-0373839-4433120,2022-04-05,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,JNE3501,JNE3501-KR-M,kurta,M,B08MYVCDB4,0,NaN,NaN,BENGALURU,KARNATAKA,560072,IN,False
41292,408-0373839-4433120,2022-04-05,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,JNE3501,JNE3501-KR-M,kurta,M,B08MYVCDB4,0,NaN,NaN,BENGALURU,KARNATAKA,560072,IN,False
79844,171-9628368-5329958,2022-05-07,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,J0329,J0329-KR-L,kurta,L,B09KXRB7JV,0,NaN,NaN,ERNAKULAM,KERALA,682017,IN,False
79845,171-9628368-5329958,2022-05-07,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,J0329,J0329-KR-L,kurta,L,B09KXRB7JV,0,NaN,NaN,ERNAKULAM,KERALA,682017,IN,False
85790,171-3249942-2207542,2022-05-03,Shipped,On the way,Amazon,Amazon.in,Expedited,SET323,SET323-KR-NP-XL,Set,XL,B09NDKKM7P,1,INR,939.0,PUNE,MAHARASHTRA,411057,IN,False
85791,171-3249942-2207542,2022-05-03,Shipped,On the way,Amazon,Amazon.in,Expedited,SET323,SET323-KR-NP-XL,Set,XL,B09NDKKM7P,1,INR,939.0,PUNE,MAHARASHTRA,411057,IN,False
86418,405-8669298-3850736,2022-05-03,Shipped,On the way,Amazon,Amazon.in,Expedited,MEN5025,MEN5025-KR-XXXL,kurta,3XL,B08YYQS8FH,1,INR,533.0,GHAZIABAD,UTTAR PRADESH,201010,IN,False
86419,405-8669298-3850736,2022-05-03,Shipped,On the way,Amazon,Amazon.in,Expedited,MEN5025,MEN5025-KR-XXXL,kurta,3XL,B08YYQS8FH,1,INR,533.0,GHAZIABAD,UTTAR PRADESH,201010,IN,False


In [19]:
#Se eliminan las filas duplicadas
df_ventas.drop_duplicates(inplace=True)

In [20]:
frecuencia = df_ventas['order_id'].value_counts()
frecuencia[frecuencia > 1].head()

order_id
403-4984515-8861958    12
171-5057375-2831560    12
404-9932919-6662730    11
403-0173977-3041148    11
408-3317403-1729937    10
Name: count, dtype: int64

Al visualizar los valores repetidos de *Order ID* llegamos a la conclusion de que se trata de varias ventas que pertenecen a la misma orden.

In [21]:
df_ventas[df_ventas['order_id']== "403-4984515-8861958"].head(3)

,order_id,date,shipping_status,delivery_status,fulfilment,sales_channel,ship_service_level,style,sku,category,size,asin,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,b2b
31737,403-4984515-8861958,2022-04-11,Shipped,On the way,Amazon,Amazon.in,Expedited,JNE3792,JNE3792-KR-XXXL,kurta,3XL,B09KXNWRSB,1,INR,432.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31738,403-4984515-8861958,2022-04-11,Shipped,On the way,Amazon,Amazon.in,Expedited,JNE3764,JNE3764-KR-XXXL,kurta,3XL,B09K3SSDSZ,1,INR,487.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31739,403-4984515-8861958,2022-04-11,Shipped,On the way,Amazon,Amazon.in,Expedited,JNE3503,JNE3503-KR-XXXL,kurta,3XL,B08RP2BV5Q,1,INR,318.0,SOLAPUR,MAHARASHTRA,413002,IN,False


In [22]:
#Filtramos por Order ID y una fecha diferente, ya que una orden puede pertencer a 1 dia especifico
df_ventas[(df_ventas['order_id']== "403-4984515-8861958") & (df_ventas['date']!="2022-04-11")].head(2)

,order_id,date,shipping_status,delivery_status,fulfilment,sales_channel,ship_service_level,style,sku,category,size,asin,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,b2b


**Tratamiento de valores Nulos**

In [23]:
#Trae todas las filas que contengan todos los valores nulos
df_ventas[df_ventas.isna().all(axis=1)]

,order_id,date,shipping_status,delivery_status,fulfilment,sales_channel,ship_service_level,style,sku,category,size,asin,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,b2b


In [24]:
columnas_con_nulos = []
for columna in df_ventas.columns:
    nulos = df_ventas[columna].isnull().sum()
    #Si la columna contiene valores nulos, se añade a la lista
    if nulos > 0: 
        columnas_con_nulos.append({'columna':columna,
                                   'total nulos':nulos})

df_nulos = pd.DataFrame(columnas_con_nulos)
df_nulos

,columna,total nulos
0,currency,7792
1,amount,7792
2,ship_city,33
3,ship_state,33
4,ship_country,33


In [25]:
df_ventas['currency'].unique()

array(['INR', nan], dtype=object)

In [26]:
#remplazo los 'nan' por 'INR' en la columna currency
df_ventas['currency'].fillna('INR',inplace=True)

<small>Analizamos la columna *amount* la cual cuenta con varios valores nulos.

A primera vista podemos observar que las filas que tienen valores 'nan' en la columna *amount* son las que contienen el valor 'Cancelled' en la columna *shipping_status*.<small>

In [27]:
df_ventas[df_ventas['amount'].isnull()].head()

,order_id,date,shipping_status,delivery_status,fulfilment,sales_channel,ship_service_level,style,sku,category,size,asin,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,b2b
8,407-5443024-5233168,2022-04-30,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,SET200,SET200-KR-NP-A-XXXL,Set,3XL,B08L91ZZXN,0,INR,NaN,HYDERABAD,TELANGANA,500008,IN,False
29,404-5933402-8801952,2022-04-30,Cancelled,Cancelled,Merchant,Amazon.in,Standard,JNE2132,JNE2132-KR-398-XXXL,kurta,3XL,B07JG3CND8,0,INR,NaN,GUWAHATI,ASSAM,781003,IN,False
65,171-4137548-0481151,2022-04-30,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,JNE3373,JNE3373-KR-XXL,kurta,XXL,B082W8RWN1,0,INR,NaN,Dahod,Gujarat,389151,IN,False
84,403-9950518-0349133,2022-04-30,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,JNE3510,JNE3510-KR-M,kurta,M,B08WPR5MCB,0,INR,NaN,HYDERABAD,TELANGANA,500072,IN,False
95,405-9112089-3379536,2022-04-30,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,JNE3405,JNE3405-KR-L,kurta,L,B081WSCKPQ,0,INR,NaN,PUNE,MAHARASHTRA,411046,IN,False


In [28]:
#Cuento las filas donde amount sea 'nan' y shipping_status sea distinto de 'cancelled'
df_ventas[(df_ventas['amount'].isnull()) & (df_ventas['shipping_status']!="Cancelled")].shape[0]

229

In [29]:
#Filas donde amount es 'nan' y shipping_status es igual a 'cancelled' y quantity es distinto de ' 0 '
df_ventas[(df_ventas['amount'].isnull()) & (df_ventas['shipping_status']=="Cancelled") & (df_ventas['quantity']!=0)].head()

,order_id,date,shipping_status,delivery_status,fulfilment,sales_channel,ship_service_level,style,sku,category,size,asin,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,b2b
12974,S02-1862183-4415901,2022-04-22,Cancelled,Cancelled,Amazon,Non-Amazon,Standard,NW008,NW008-ST-CP-XXL,Set,XXL,B0922V4R3Z,1,INR,NaN,Surat,Gujarat,394210,IN,False


In [30]:
cantidad1 = df_ventas[(df_ventas['amount'].isnull()) & (df_ventas['quantity']==0)].shape[0]
print(f"Cantidad de filas con Amount nulo y Quantity igual a 0: {cantidad1}")

cantidad2 = df_ventas[(df_ventas['amount'].isnull()) & (df_ventas['quantity']>0)].shape[0]
print(f"Cantidad de filas con Amount nulo y Quantity mayor a 0: {cantidad2}")

Cantidad de filas con Amount nulo y Quantity igual a 0: 7668
Cantidad de filas con Amount nulo y Quantity mayor a 0: 124


<small>En conclusion se identifico que la mayoria de los valores nulos en *amount* se asocia con pedidos cuyo estado de envío es "Cancelled". Cabe destacar que también se encontraron valores nulos en la columna *amount* para pedidos que no están en estado "Cancelled"

Vemos que:
- Hay 1 fila donde *amount* contiene 'Nan', el estado del envio es 'Cancelled' y *quantity* es mayor que 0. <br>
- Hay 7668 filas donde *amount* contiene 'Nan' y *quantity* es igual a 0.<br>
- Hay 124 filas donde *amount* contiene 'Nan' y *quantity* es mayor que 0.<br>



Lo que haremos ahora es ver si el precio de los productos esta calculado de acuerdo a la cantidad o si solo esta el precio individual, de esta forma sabremos como imputar los datos.<small>

In [31]:
df_ventas['quantity'].value_counts().head()

quantity
1    115777
0     12804
2       341
3        32
4         9
Name: count, dtype: int64

In [32]:
fila_sku_amount = df_ventas[df_ventas['sku'] == 'JNE3797-KR-L'][['sku','quantity','amount']].head(1)
sku = fila_sku_amount['sku'].iloc[0]
precio = fila_sku_amount['amount'].iloc[0]
print(f"Sku de producto: {sku}, quantity: 1, amount: {precio}")

fila_quantity_amount =  df_ventas[(df_ventas['sku'] == 'JNE3797-KR-L') & (df_ventas['quantity']>1)][['sku','quantity', 'amount']]
sku = fila_quantity_amount['sku'].iloc[0]
cantidad = fila_quantity_amount['quantity'].iloc[0]
precio = fila_quantity_amount['amount'].iloc[0]
print(f"Sku de producto: {sku}, quantity: {cantidad}, amount: {precio}")


Sku de producto: JNE3797-KR-L, quantity: 1, amount: 725.0
Sku de producto: JNE3797-KR-L, quantity: 2, amount: 1470.0


Podemos observar que los precios (amount) que se encuentran en las filas estan calculados segun la cantidad de productos y no de manera individual.

Entonces primero modificaremos las filas de la columna *quantity* que contengan 0 como valor.<br>
Aquellas filas donde *amount* sea Null y *quantity* sea 0, se imputaran con la mediana y el valor correspondiente a esa unidad, apartir del *sku* correspondiente.

In [38]:
print(f"Promedio de quantity: {round(np.mean(df_ventas['quantity']!=0),3)}")
print(f"Mediana de quantity: {np.median(df_ventas['quantity']!=0)}")


Promedio de quantity: 0.96
Mediana de quantity: 1.0


In [34]:
#Ingreso 1 en las filas donde quantity contiene 0 y amount contiene nulos
df_ventas['quantity'] = np.where((df_ventas['quantity'] == 0) & (df_ventas['amount'].isna()), 1, df_ventas['quantity'])

In [39]:
#Verifico que no hayan quedado 'ceros' en las filas donde amount contiene 'nan' y quantity contenia 0
cantidad_ceros = df_ventas[(df_ventas['quantity'] == 0) & (df_ventas['amount'].isna())].shape[0]
cantidad_ceros

0

In [36]:
df_ventas[(df_ventas['sku']=='JNE3797-KR-L') & (df_ventas['quantity']!=0) & (df_ventas['amount'].isnull())].head()

,order_id,date,shipping_status,delivery_status,fulfilment,sales_channel,ship_service_level,style,sku,category,size,asin,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,b2b
24587,406-6591982-1553107,2022-04-15,Cancelled,Cancelled,Amazon,Amazon.in,Expedited,JNE3797,JNE3797-KR-L,Western Dress,L,B09SDXFFQ1,1,INR,NaN,ALUVA,KERALA,683104,IN,False
33677,408-1220954-0396302,2022-04-10,Cancelled,Cancelled,Merchant,Amazon.in,Standard,JNE3797,JNE3797-KR-L,Western Dress,L,B09SDXFFQ1,1,INR,NaN,PRODDATUR,ANDHRA PRADESH,516360,IN,False
34681,403-3827542-4301913,2022-04-10,Cancelled,Cancelled,Merchant,Amazon.in,Standard,JNE3797,JNE3797-KR-L,Western Dress,L,B09SDXFFQ1,1,INR,NaN,HYDERABAD,TELANGANA,500070,IN,False
34757,405-0612507-9398758,2022-04-09,Cancelled,Cancelled,Merchant,Amazon.in,Standard,JNE3797,JNE3797-KR-L,Western Dress,L,B09SDXFFQ1,1,INR,NaN,HYDERABAD,TELANGANA,500068,IN,False
34766,405-3069909-6569136,2022-04-09,Cancelled,Cancelled,Merchant,Amazon.in,Standard,JNE3797,JNE3797-KR-L,Western Dress,L,B09SDXFFQ1,1,INR,NaN,HYDERABAD,TELANGANA,500068,IN,False
